# Original

In [16]:
import os
import cv2
import numpy as np
import pickle
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as pnp
from math import ceil, log2

# === Setup ===
dataset_path = "D:/datasets/PetImages"
categories = ["Cat", "Dog"]
num_images_per_category = 1000

# === Feature Extraction using SIFT ===
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros(128)
    return descriptors.mean(axis=0)

# === Load Dataset in Parallel ===
def process_image(category, image_file):
    image_path = os.path.join(dataset_path, category, image_file)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is not None:
        feat = extract_features(image)
        return feat, category
    return None, None

all_features, labels = [], []
for category in categories:
    print(f"Processing {category} images...")
    path = os.path.join(dataset_path, category)
    image_files = [f for f in os.listdir(path) if f.endswith(".jpg")][:num_images_per_category]

    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda f: process_image(category, f), image_files), total=len(image_files)))
    
    for feat, label in results:
        if feat is not None:
            all_features.append(feat)
            labels.append(label)

# === Standardization + PCA ===
scaler = StandardScaler()
scaled = scaler.fit_transform(all_features)

pca = PCA(0.95)  # retain 95% variance
reduced = pca.fit_transform(scaled)

# === Quantum Configuration ===
original_dim = reduced.shape[1]
n_qubits = int(ceil(log2(original_dim)))
dim = 2**n_qubits

# Pad to match 2^n
def pad(vec, target_dim):
    if len(vec) < target_dim:
        return np.pad(vec, (0, target_dim - len(vec)))
    else:
        return vec[:target_dim]

# Quantum Device
dev = qml.device("lightning.qubit", wires=n_qubits)

# QAOA Layer
def qaoa_layer(gamma, beta, wires):
    for i in range(len(wires)):
        qml.CZ(wires=[wires[i], wires[(i+1)%len(wires)]])
    for i in range(len(wires)):
        qml.RX(gamma, wires=i)
        qml.RZ(beta, wires=i)

# QNode
@qml.qnode(dev)
def quantum_model(x, gamma=0.5, beta=0.5):
    qml.AmplitudeEmbedding(x / np.linalg.norm(x), wires=range(n_qubits), normalize=True, pad_with=0.0)
    qaoa_layer(gamma, beta, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# === Encode All with Quantum Circuit ===
quantum_outputs = []
print("Quantum encoding in progress...")
for vec in tqdm(reduced):
    vec_padded = pad(vec, dim)
    vec_pnp = pnp.array(vec_padded, requires_grad=False)
    result = quantum_model(vec_pnp)
    quantum_outputs.append(result)

# === Save Output ===
final_data = {
    "quantum_encoded": quantum_outputs,
    "labels": labels
}
with open("quantum_real_time_data.pkl", "wb") as f:
    pickle.dump(final_data, f)

print(" Quantum encoding complete with dynamic qubits and padded Amplitude Embedding!")

Processing Cat images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:36<00:00, 27.32it/s]


Processing Dog images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:47<00:00, 21.01it/s]


Quantum encoding in progress...


100%|█████████████████████████████████████████████████████████████████████████████| 1992/1992 [00:10<00:00, 189.54it/s]

✅ Quantum encoding complete with dynamic qubits and padded Amplitude Embedding!


In [17]:
import pickle

# Load the quantum encoded data
with open("quantum_real_time_data.pkl", "rb") as f:
    data = pickle.load(f)

# Print the first 5 quantum encoded vectors and their labels
print(" Quantum Encoded Outputs (First 5):")
for i in range(5):
    print(f"Sample {i+1}:")
    print("  Label  :", data["labels"][i])
    print("  Encoding:", data["quantum_encoded"][i])
    print()


🔹 Quantum Encoded Outputs (First 5):
Sample 1:
  Label  : Cat
  Encoding: [array(0.72441723), array(0.3273361), array(0.42893438), array(-0.40457318), array(0.57103061)]

Sample 2:
  Label  : Cat
  Encoding: [array(0.84443226), array(0.78705741), array(0.6994629), array(0.43953125), array(0.71539102)]

Sample 3:
  Label  : Cat
  Encoding: [array(0.72123709), array(0.81350795), array(-0.55592995), array(0.57628599), array(0.50335103)]

Sample 4:
  Label  : Cat
  Encoding: [array(0.83038269), array(0.69308658), array(0.20428599), array(0.45211841), array(0.01848554)]

Sample 5:
  Label  : Cat
  Encoding: [array(0.78047408), array(0.65008033), array(0.6198495), array(0.48470764), array(0.36771226)]



# Full Frame Image size and Alterable Qubits

In [18]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as pnp

In [19]:
dataset_path = "D:/datasets/PetImages"
categories = ["Cat", "Dog"]
num_images_per_category = 1000
all_features = []
labels = []

In [20]:
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros((128,))
    return descriptors.mean(axis=0)  # returns a 128-dim vector


In [21]:
for category in categories:
    path = os.path.join(dataset_path, category)
    image_files = [f for f in os.listdir(path) if f.endswith(".jpg")][:num_images_per_category]
    print(f"Encoding {category} images...")

    for image_file in tqdm(image_files):
        try:
            image_path = os.path.join(path, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue
            features = extract_features(image)
            all_features.append(features)
            labels.append(category)
        except Exception as e:
            print(f"Error processing {image_file}: {e}")

Encoding Cat images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:04<00:00, 15.61it/s]


Encoding Dog images...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:08<00:00, 14.65it/s]


In [22]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(all_features)

# Dynamic number of PCA components (limit to 8)
n_components = min(8, scaled_features.shape[1])
pca = PCA(n_components=n_components)
reduced_features = pca.fit_transform(scaled_features)

In [23]:
n_qubits = n_components
dev = qml.device("default.qubit", wires=n_qubits)

def qaoa_layer(gamma, beta, wires):
    for i in range(len(wires)):
        qml.CZ(wires=[wires[i], wires[(i + 1) % len(wires)]])
    for i in range(len(wires)):
        qml.RX(2 * gamma, wires=i)
        qml.RZ(2 * beta, wires=i)

In [24]:
@qml.qnode(dev)
def quantum_model(x, gamma=0.5, beta=0.5):
    padded = np.zeros(2**n_qubits)
    norm = np.linalg.norm(x)
    padded[:len(x)] = x / norm if norm != 0 else x
    qml.AmplitudeEmbedding(features=padded, wires=range(n_qubits), normalize=True)
    qaoa_layer(gamma, beta, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))


In [25]:
quantum_outputs = []

print("Quantum encoding in progress...")
for vec in tqdm(reduced_features):
    vec = pnp.array(vec, requires_grad=False)
    result = quantum_model(vec)
    quantum_outputs.append(result)

Quantum encoding in progress...


100%|█████████████████████████████████████████████████████████████████████████████| 1992/1992 [00:18<00:00, 109.32it/s]


In [29]:
final_data = {
    "quantum_encoded": quantum_outputs,
    "labels": labels
}

with open("quantum_real_time_data.pkl", "wb") as f:
    pickle.dump(final_data, f)

print(" Pipeline complete with adaptive qubit encoding and no resizing.")

 Pipeline complete with adaptive qubit encoding and no resizing.


In [28]:
# 🖨️ Print Head
print("\n Preview of Encoded Quantum Features:")
for i in range(5):
    print(f"Label: {labels[i]}, Quantum Output: {quantum_outputs[i]}")


 Preview of Encoded Quantum Features:
Label: Cat, Quantum Output: 0.5403023058681407
Label: Cat, Quantum Output: 0.5403023058681409
Label: Cat, Quantum Output: 0.5403023058681404
Label: Cat, Quantum Output: 0.5403023058681408
Label: Cat, Quantum Output: 0.5403023058681403


# ORB VERSION

In [48]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as pnp

In [49]:
dataset_path = "D:/datasets/PetImages"
categories = ["Cat", "Dog"]
num_images_per_category = 1000
all_features = []
labels = []

In [50]:
# 🎯 Feature extraction using ORB (Star Feature Alternative)
def extract_features(image):
    orb = cv2.ORB_create(nfeatures=128)
    keypoints, descriptors = orb.detectAndCompute(image, None)
    if descriptors is None:
        return np.zeros((128,))
    return descriptors.mean(axis=0)

In [51]:
for category in categories:
    path = os.path.join(dataset_path, category)
    image_files = [f for f in os.listdir(path) if f.endswith(".jpg")][:num_images_per_category]
    print(f"Encoding {category} images...")

    for image_file in tqdm(image_files):
        try:
            image_path = os.path.join(path, image_file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue
            features = extract_features(image)
            all_features.append(features)
            labels.append(category)
        except Exception as e:
            print(f"Error processing {image_file}: {e}")

Encoding Cat images...


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 126.03it/s]


Encoding Dog images...


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 102.64it/s]


In [52]:
fixed_length = 128  # or any fixed dimension you choose

def pad_or_truncate(feature, length=fixed_length):
    if feature is None:
        return np.zeros(length)
    if len(feature) >= length:
        return feature[:length]
    else:
        return np.pad(feature, (0, length - len(feature)), mode='constant')

# Apply fix to all features
fixed_features = np.array([pad_or_truncate(f) for f in all_features])

# Now scale and reduce
scaler = StandardScaler()
scaled_features = scaler.fit_transform(fixed_features)

n_components = min(8, scaled_features.shape[1])
pca = PCA(n_components=n_components)
reduced_features = pca.fit_transform(scaled_features)


In [53]:
n_qubits = n_components
dev = qml.device("default.qubit", wires=n_qubits)

def qaoa_layer(gamma, beta, wires):
    for i in range(len(wires)):
        qml.CZ(wires=[wires[i], wires[(i + 1) % len(wires)]])
    for i in range(len(wires)):
        qml.RX(2 * gamma, wires=i)
        qml.RZ(2 * beta, wires=i)

In [54]:
@qml.qnode(dev)
def quantum_model(x, gamma=0.5, beta=0.5):
    padded = np.zeros(2**n_qubits)
    norm = np.linalg.norm(x)
    padded[:len(x)] = x / norm if norm != 0 else x
    qml.AmplitudeEmbedding(features=padded, wires=range(n_qubits), normalize=True)
    qaoa_layer(gamma, beta, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

In [55]:
quantum_outputs = []

print("Quantum encoding in progress...")
for vec in tqdm(reduced_features):
    vec = pnp.array(vec, requires_grad=False)
    result = quantum_model(vec)
    quantum_outputs.append(result)

Quantum encoding in progress...


100%|██████████████████████████████████████████████████████████████████████████████| 1992/1992 [00:22<00:00, 89.37it/s]


In [56]:
final_data = {
    "quantum_encoded": quantum_outputs,
    "labels": labels
}

with open("quantum_real_time_data.pkl", "wb") as f:
    pickle.dump(final_data, f)

print(" Pipeline complete with adaptive qubit encoding and no resizing.")

 Pipeline complete with adaptive qubit encoding and no resizing.


In [57]:
# 🖨️ Print Head
print("\n Preview of Encoded Quantum Features:")
for i in range(5):
    print(f"Label: {labels[i]}, Quantum Output: {quantum_outputs[i]}")


 Preview of Encoded Quantum Features:
Label: Cat, Quantum Output: 0.5403023058681404
Label: Cat, Quantum Output: 0.5403023058681404
Label: Cat, Quantum Output: 0.5403023058681407
Label: Cat, Quantum Output: 0.5403023058681405
Label: Cat, Quantum Output: 0.5403023058681402
